from IPython.core.magic import register_cell_magic, cell_magic

@register_cell_magic
def parallel(line, cell):
    from os import environ
    parallel = bool(environ.get('PARALLEL', False))
    if parallel:
        if '--except' in line:
            print('Skipping cell when in parallel')
            pass
        else:
            get_ipython().run_cell_magic('px', '--local', cell)
    else:
        if '--only' in line:
            print('Skipping cell when not in parallel')
            return
        else:
            get_ipython().run_cell(cell)
            
@register_cell_magic
def comment(line, cell):
    '''Comment a cell.'''
    print('Skipping cell.')
    pass

In [ ]:
%%parallel --only
from IPython import parallel
c = parallel.Client()
view = c.load_balanced_view()

In [ ]:
%%parallel --except
%matplotlib notebook

In [ ]:
%%parallel
%load_ext Cython
%load_ext autoreload

In [ ]:
%%parallel
%%cython
cimport cython
import numpy as np
cimport numpy as np
#from cython.parallel import prange

ctypedef double DTYPE_t

@cython.boundscheck(False)

def compute_inertia_tensor_1(np.ndarray[DTYPE_t, ndim=1] mass, np.ndarray[DTYPE_t, ndim=2] pos, no_divide_by_mass=False):
    '''Takes the mass and positions of particles and return the inertia tensor'''
    cdef int i, j
    cdef np.ndarray[DTYPE_t, ndim=2] I_t, tmp_pos
    cdef np.ndarray[DTYPE_t, ndim=1] means
    cdef DTYPE_t tmp, mtot
    
    I_t = np.zeros((3, 3), dtype=np.double)
    tmp_pos = np.zeros_like(pos)
    mtot = np.sum(mass)
    
    # remove mean from positions
    means = np.mean(pos, 0)
    
    for j in range(pos.shape[0]):
        for i in range(pos.shape[1]):
            tmp_pos[j, i] = pos[j, i] - means[i]
    
    for i in range(3):
        for j in range(i, 3):
            tmp = 0
            for k in range(pos.shape[0]):
                tmp = tmp + mass[k]*tmp_pos[k, i]*tmp_pos[k, j]
            I_t[i, j] = tmp
            I_t[j, i] = tmp
    if no_divide_by_mass:
        return I_t
    else:
        return I_t/mtot

def compute_inertia_tensor(mass, pos, no_divide_by_mass=False):
    tmp = pos - np.mean(pos, 0)
    if no_divide_by_mass:
        return np.dot(mass*tmp.T, tmp)
    else:
        return np.dot(mass*tmp.T, tmp) / np.sum(mass)

def project_points(x, y, z, a, b, c):
    """
    Projects the points with coordinates x, y, z onto the plane
    defined by a*x + b*y + c*z = 1
    
    From http://stackoverflow.com/questions/17836880/orthogonal-projection-with-numpy
    """
    vector_norm = a*a + b*b + c*c
    normal_vector = np.array([a, b, c]) / np.sqrt(vector_norm)
    point_in_plane = np.array([a, b, c]) / vector_norm

    points = np.column_stack((x, y, z))
    points_from_point_in_plane = points - point_in_plane
    proj_onto_normal_vector = np.dot(points_from_point_in_plane,
                                     normal_vector)
    proj_onto_plane = (points_from_point_in_plane -
                       proj_onto_normal_vector[:, None]*normal_vector)

    return point_in_plane + proj_onto_plane

cdef DTYPE_t correct(DTYPE_t el):
    '''Realign the particles when they're too far appart'''
    if el < 0.5:
        return el
    else:
        return el - 1.0
v_correct = np.vectorize(correct)

@cython.boundscheck(False)
def correct_particles(np.ndarray[DTYPE_t, ndim=2] particles):
    cdef int i, j
    cdef np.ndarray[DTYPE_t, ndim=1] maxis, minis
    cdef np.ndarray[DTYPE_t, ndim=2] ret
    maxis = np.max(particles, 0)
    minis = np.min(particles, 0)
    
    ret = particles
    for i in range(3):
        if maxis[i] - minis[i] > 0.5:
            print('Reshaping %s' % i)
            for j in range(particles.shape[0]):
                if (ret[j, i] > 0.5):
                    ret[j, i] = ret[j, i] - 1.0
    return ret

@cython.boundscheck(False)
cdef int binary_search(np.ndarray[int, ndim=1] a, int x, int left, int right):
    cdef int lo, hi, mid, midval
    lo = left
    hi = right
    while lo < hi:
        mid = (lo+hi)/2
        midval = a[mid]
        if midval < x:
            lo = mid+1
        elif midval > x: 
            hi = mid
        else:
            return mid
    return -1
        
def quicksearch_2(np.ndarray array, np.ndarray elements):
    '''Searches elements in array, where array and elements
    are both sorted'''
    cdef int left, right, pos
    positions = np.zeros(len(elements), dtype=int)
    left = 0
    right = len(array)
    for i in range(len(elements)):
        pos = binary_search(array, elements[i], left, right)
        if pos >-1:
            left = pos
        positions[i] = pos
    
    return positions

def quicksearch(np.ndarray[int, ndim=1] array, np.ndarray[int, ndim=1] elements):
    '''Returns the index of the elements in the array, asserting elements and array are ordered.'''
    cdef int i, ubound
    cdef np.ndarray[long, ndim=1] res, res1
    
    res = np.searchsorted(array, elements)
    
    ubound = len(array)

    for i in range(len(res)):
        if res[i] == 0 and elements[i] != array[res[i]]:
            res[i] = -1
        elif res[i] == ubound:
            res[i] = -1
    return res

def search_mask(np.ndarray[int, ndim=1] array, np.ndarray[int, ndim=1] elements):
    '''Returns the index of the elements in the array, asserting elements and array are ordered.'''
    return np.where(quicksearch(array, elements) >= 0, True, False)

def quicksearch_1(np.ndarray[int, ndim=1] array, np.ndarray[int, ndim=1] elements):
    cdef int i, ubound
    cdef np.ndarray[long, ndim=1] res0, res1
    res0 = np.searchsorted(array, elements, side='left')
    res1 = np.searchsorted(array, elements, side='right')
    return np.where(res1-res0 == 1, res0, -1)

def in_region(np.ndarray center, DTYPE_t radius):
    cdef DTYPE_t r2
    r2 = radius**2
    def tmp(np.ndarray[DTYPE_t, ndim=2] pts):
        cdef np.ndarray pos2
        pos2 = np.sum(np.power(pts, 2) - center, 1)
        return pos2 < r2
    return tmp

In [ ]:
x = np.array([[1, 1, 1],
              [1, 2, 3]], dtype=np.float)
m = np.array([0.1, 0.1, 0.1])
x = np.random.rand(100, 3)
m = np.random.rand(100)
%timeit compute_inertia_tensor(m, x)
%timeit compute_inertia_tensor_1(m, x)
compute_inertia_tensor(m, x), compute_inertia_tensor_1(m, x), compute_inertia_tensor(m, x)-compute_inertia_tensor_1(m, x)<1e-12

In [ ]:
%%parallel
%autoreload
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import tools as t
import pymses
import itertools
from tqdm import tqdm

import scipy.linalg as linalg
from scipy.interpolate import griddata
from scipy.stats import gaussian_kde

from sklearn.neighbors import KernelDensity

#from cython_module import *

from mpl_toolkits.mplot3d import Axes3D

plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['figure.dpi'] = 72
matplotlib.style.use('ggplot')

In [ ]:
%%comment
%%px --local
def compute_inertia_tensor(mass,
                           pos):
    '''Takes the mass and positions of particles and return the inertia tensor'''
    #cdef int i, j
    #cdef np.ndarray[DTYPE_t, ndim=2] tmp, I_t
    I_t = np.zeros((3, 3))
    for i in range(3):
        for j in range(i, 3):
            tmp = np.sum(mass*pos[:, i]*pos[:, j]) / np.sum(mass)
            I_t[i, j] = tmp
            I_t[j, i] = tmp
    return I_t

# Get inertia data

In [ ]:
#%%px --local
halo_inertia = pd.read_csv('lists/halo.00002.inertia_tensor.dat', delim_whitespace=True).set_index('halo_id')

In [ ]:
halo_inertia.describe()

# List of halos

In [ ]:
#%%px --local
halo_list = pd.read_csv('lists/list_halo.dat',
                        delim_whitespace=True,
                        skiprows=1,
                        names=['id', 'level', 'mass', 'x', 'y', 'z', 'r']).set_index('id')

In [ ]:
halo_list[halo_list.mass > 1e12].head()

# Galaxy dynamics

In [ ]:
#%%px --local

unit, ncol, nrow = t.io.read_list_header('lists/list_kingal_00782.dat')
gal_dynamics = pd.DataFrame(t.io.read_list_data_reals(unit, ncol, nrow),
                            columns=['id', 'vtheta', 'dvx', 'dvy', 'dvz', 'mass', 'x', 'y', 'z']).set_index('id')

gd = gal_dynamics
gal_dynamics['sigma_over_vtheta'] = 1./3*np.sqrt((gd.dvx**2 + gd.dvy**2 + gd.dvz**2)) / np.abs(gd.vtheta)
del gd

In [ ]:
gal_dynamics.head()

# Galaxy to halo

In [ ]:
#%%px --local
association = pd.read_csv('lists/associated_halogal_782.dat', delim_whitespace=True, skiprows=1,
                          names=['halo_id', 'level', 'halo_mass', 'gal_id', 'gal_mass']).set_index('halo_id')

In [ ]:
association.head()

# Halo to cpu

In [ ]:
#%%px --local
filename = 'lists/halo_to_cpu.00002.m<1e12.dat'
with open(filename, 'r') as f:
    rows, cols = [int(e) for e in f.readline().replace('\n', '').split()]
    names = ['halo_id'] + ['cpu_%i' % cpu for cpu in range(1, cols+1)] 

halo_to_cpu = pd.read_csv(filename, delim_whitespace=True, engine='c', skiprows=1,
                          names=names).set_index('halo_id')

In [ ]:
halo_to_cpu[halo_to_cpu.cpu_1 > 0].head()

# Get brick

In [ ]:
#%%px --local
halos = t.io.read_brick('/data52/Horizon-AGN/TREE_DM_celldx2kpc_SC0.9r/tree_bricks782', low_mem=True, preload=True)

# Test with 1 halo of mass ~1e12

In [ ]:
halo_list[halo_list.mass > 1e12].head()

In [ ]:
id = 17316
cpus = [cpu for cpu in list(halo_to_cpu.loc[id]) if cpu > 0]
halo_to_cpu.loc[id]

In [ ]:
raw_parts = pd.DataFrame()
for cpu in cpus:
    print('Reading cpu', cpu)
    filename = '/data52/Horizon-AGN/OUTPUT_DIR/output_00002/part_00002.out{:0>5}'.format(cpu)
    _,_,_, _tmp = t.io.read_particles(filename)
    _tmp['cpu'] = cpu
    raw_parts = raw_parts.append(_tmp)
raw_parts = raw_parts.set_index('ids')
raw_parts.head()

In [ ]:
data_mask = np.unique([i for i in quicksearch(np.array(raw_parts.index, dtype=np.int32), halos[id-1]['members']) if i >= 0])
data_mask
#[filter(s) for _, s in tqdm(raw_parts.iterrows())]

In [ ]:
xyz = correct_particles(np.copy(np.array(raw_parts[['x', 'y', 'z']])))
raw_parts.x = xyz[:, 0]
raw_parts.y = xyz[:, 1]
raw_parts.z = xyz[:, 2]
raw_parts.describe()

In [ ]:
raw_parts_2 = raw_parts.iloc[data_mask]
print('Found', raw_parts_2.x.size, 'particles, expected', len(halos[id-1]['members']))
raw_parts_2.head()
means = np.array(raw_parts_2[['x', 'y', 'z']].mean())
sigmas = np.array(raw_parts_2[['x', 'y', 'z']].std())
del raw_parts
raw_parts_2.describe()

In [ ]:
ro = pymses.RamsesOutput('/data52/Horizon-AGN/OUTPUT_DIR/', 2)
ro.verbose = True
parts = ro.particle_source(["pos", "mass", "id"])

### Get the cpus containing the halo

In [ ]:
print('Cpus in box', means-2*sigmas, means+2*sigmas)
cpus = np.unique([c 
        for c in t.misc.get_cpu_list(means - 2*sigmas, means + 2*sigmas, ro.info['levelmax'], ro.info['dom_decomp_Hilbert_keys'])
        if c > 0])
print('%f cpus', len(cpus))

In [ ]:
smax = np.max(sigmas)
f = 5
sps = []
for i in range(3):
    if (means[i] - f*smax < 0):
        print(means[i] - f*smax )
        tmp_means = np.copy(means)
        tmp_means[i] += 1
        sps.append(pymses.utils.regions.Sphere(tmp_means, f*smax))
    if (means[i] + f*smax > 1.):
        print(means[i] + f*smax )
        tmp_means = np.copy(means)
        tmp_means[i] -= 1
        sps.append(pymses.utils.regions.Sphere(tmp_means, f*smax))
sps.append(pymses.utils.regions.Sphere(means, f*smax))

idFilter = pymses.filters.PointIdFilter(list(halos[id-1]['members']), parts)
regs = [pymses.filters.RegionFilter(sp, idFilter) for sp in sps]
#regs_id = [pymses.filters.PointIdFilter(list(halos[id-1]['members']), r) for r in regs]
raw_parts = pd.DataFrame()
for reg in tqdm(regs):
    for dset in reg.iter_dsets():
        xyz = dset.points
        masses = dset['mass']
        tmp = pd.DataFrame({'ids': dset['id'], 'm': dset['mass'], 'x': xyz[:, 0], 'y': xyz[:, 1], 'z': xyz[:, 2]})
        raw_parts = raw_parts.append(tmp)
        del tmp
parts = raw_parts.set_index('ids')

In [ ]:
print(len(halos[id-1]['members']), parts['m'].size)
raw_parts.describe()
xyz = correct_particles(np.array(parts[['x', 'y', 'z']]))
parts.x = xyz[:, 0]
parts.y = xyz[:, 1]
parts.z = xyz[:, 2]
np.mean(xyz, 0), np.min(xyz, 0), np.max(xyz, 0)

In [ ]:
print(np.mean(xyz, 0))
res.loc[id:id]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
I_t = compute_inertia_tensor(parts.m.as_matrix(), parts.as_matrix(['x', 'y', 'z']))
eigvals, eigvects = np.linalg.eigh(I_t)
each = 10
ax.scatter3D(raw_parts.x[::each], raw_parts.y[::each], raw_parts.z[::each])
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
mx, my, mz = raw_parts[['x', 'y', 'z']].mean()
for i in range(3):
    ax.plot([mx, mx+np.sqrt(eigvals[i])*eigvects[i, 0]],
            [my, my+np.sqrt(eigvals[i])*eigvects[i, 1]],
            [mz, mz+np.sqrt(eigvals[i])*eigvects[i, 2]], c='black')
print(eigvals)
print(eigvects)
eigvects[0, 0], eigvects[1, 0], res.loc[id:id][['meanx', 'meany', 'meanz']]

### Project on eigenvectors

In [ ]:
def project(x, y, z):
    print(x, y, z)
    return np.dot(eigvects, np.array((x, y, z)))
vproject = np.vectorize(project)
#vproject(parts[['x', 'y', 'z']][:100], 3)
e0, e1, e2 = np.dot(eigvects.T, parts[['x', 'y', 'z']].T)
parts['e0'], parts['e1'], parts['e2'] = e0, e1, e2

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5), squeeze=True)
proj = {}
each = 1
for i, xy, l in zip([0, 1, 2], ['e0 e1', 'e1 e2', 'e0 e2'], np.sqrt([eigvals[2], eigvals[1], eigvals[0]])):
    ax = axes[i]
    xname, yname = xy.split(' ')
    ax.scatter(parts[xname][::each], parts[yname][::each], alpha=0.5)
    m0, m1 = parts[[xname, yname]].mean()
    l0, l1 = np.sqrt(eigvals[[int(xname.replace('e', '')), int(yname.replace('e', ''))]])
    ax.plot([m0, m0+l0], [m1, m1])
    ax.plot([m0, m0], [m1, m1+l1])
    ax.axis('equal')
    ax.grid('on')
    ax.set_xlabel(xname)
    ax.set_ylabel(yname)
    ax.set_title(r'$\sqrt{\lambda} ='+'{}$'.format(l))
plt.tight_layout()

## Estimate density using KDE
Take a look at https://jakevdp.github.io/blog/2013/12/01/kernel-density-estimation/ for details

# Compute the Inertia tensor

# Dump

In [ ]:
import pickle as p
with open('.tmp/dump', 'w') as f:
    f.dump([halo_to_cpu, halos, ])

# Inverting  halo→cpu to cpu→halo

In [ ]:
cpu_to_halo = {}
for i in range(1, 4096+1):
    cpu_to_halo[i] = set()
for halo_i, cpus in tqdm(halo_to_cpu.iterrows()):
    for _, cpu in cpus.iteritems():
        if (cpu > 0):
            cpu_to_halo[cpu].add(halo_i)

# Computing inertia tensor for all halos (this may take a long time…)

In [ ]:
#%%px --local
ro = pymses.RamsesOutput('/data52/Horizon-AGN/OUTPUT_DIR/', 2)
ro.verbose = False
ro_parts = ro.particle_source(["pos", "mass", "id"])

In [ ]:
#%%px --local
step = 2
# Compute our meshgrid
txgrid = np.linspace(0, 1, step+1)
tygrid = np.linspace(0, 1, step+1)
tzgrid = np.linspace(0, 1, step+1)

xgrid = (txgrid[1:]+txgrid[:-1])/2
ygrid = (tygrid[1:]+tygrid[:-1])/2
zgrid = (tzgrid[1:]+tzgrid[:-1])/2

X, Y, Z = np.meshgrid(xgrid, ygrid, zgrid)

# Create a list of permutations 
comb_list = list(itertools.product(range(step), range(step), range(step)))

def get_halo_in_cpus(cpus):
    '''Return the halos that are contained in the cpus.'''
    halo_set = set()
    for cpu in cpus:
        halo_set.update(cpu_to_halo[cpu])
    return halo_set

In [ ]:
halo_read = set()
def compute((i, j, k)):
    halo_inertia = pd.DataFrame(columns=['id', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz',
                                         'complete', 'meanx', 'meany', 'meanz', 'nparts']).set_index('id')

    center = np.array([X[i,j,k], Y[i,j,k], Z[i,j,k]])

    region = pymses.filters.RegionFilter(pymses.utils.regions.Cube(center, 1.1/step), ro_parts)
    dset   = region.flatten()
    
    print('Computing for region', region.region.get_bounding_box())

    #for halo_i, halo in tqdm(_tmp_h_list.iterrows()):
    for halo_i in tqdm(get_halo_in_cpus(region._data_list), nested=True):
        try:
            members = halos[halo_i-1]['members']
        except:
            print('Exception')
            continue
                        
        if halo_i in halo_read:
            #print('Skipping already fully read halo %s' % halo_i)
            continue
        
        #         region_plus_id = pymses.filters.PointIdFilter(list(halos[id-1]['members']), dset_0)
        #         dset = region_plus_id.flatten()

        ids = np.unique([i for i in quicksearch(dset['id'], members) if i >=0])
        if len(ids) == 0:
            print('Empty !')
            continue
        pts = dset.points[ids]
        xyz = np.array(correct_particles(pts), dtype=np.float)
        masses = np.array(dset['mass'][ids], dtype=np.float)
        #         xyz = correct_particles(dset.points)
        #         masses = dset['mass']
        means = np.mean(xyz, 0)
        #print(np.mean(xyz, 0))
        
        halo_inertia.at[halo_i, 'meanx'] = means[0]
        halo_inertia.at[halo_i, 'meany'] = means[1]
        halo_inertia.at[halo_i, 'meanz'] = means[2]
        halo_inertia.at[halo_i, 'stdx'] = np.std(xyz[:, 0])
        halo_inertia.at[halo_i, 'stdy'] = np.std(xyz[:, 1])
        halo_inertia.at[halo_i, 'stdz'] = np.std(xyz[:, 2])
        halo_inertia.at[halo_i, 'nparts'] = len(masses)
        
        if len(masses) == len(halos[halo_i-1]['members']):
            halo_inertia.at[halo_i, 'complete'] = True
            halo_read.add(halo_i)
        else:
            halo_inertia.at[halo_i, 'complete'] = False
            #print('Halo {} is incomplete, missing {:.2}%'.format(
            #        halo_i, (100.*(len(halos[halo_i-1]['members']) - len(masses)) / (len(masses)))))


        I_t = compute_inertia_tensor(masses, xyz)
        l = ['x', 'y', 'z']
        for i in range(3):
            for j in range(i, 3):
                halo_inertia.at[halo_i, l[i]+l[j]] = np.float(I_t[i,j])

    return halo_inertia

In [ ]:
p_results_p = []
for ijk in tqdm(comb_list):
    p_results_p.append(compute(ijk))

p_results_p = view.map_sync(compute, comb_list[:5])

In [ ]:
print(len(p_results_p))
res = pd.DataFrame()
for p in p_results_p:
    res = res.append(p)

In [ ]:
#%%comment
import pickle as pickle
with open('halo_inertia_m_mean.dump', 'w') as f:
    pickle.dump(p_results_p, f)

In [ ]:
%%comment
import pickle as pickle
with open('halo_inertia_m_mean.dump', 'r') as f:
    p_results_p = pickle.load(f)

In [ ]:
%%comment
import pickle as pickle

with open('inertia_m_mean.pickle', 'w') as f:
    pickle.dump(res, f)

# Reload output

In [ ]:
%%comment
import pickle as pickle

with open('inertia_m_mean.pickle', 'r') as f:
    res = pickle.load(f).astype(np.float)

In [ ]:
res.describe(), halo_list[halo_list.mass > 1e12].describe()

# Compute the eigenvalues + vectors

In [ ]:
for halo_i, l in tqdm(res.iterrows()):
    I_t = np.array([[l.xx, l.xy, l.xz],
                    [l.xy, l.yy, l.yz],
                    [l.xz, l.yz, l.zz]])
    lambdas = np.linalg.eigvalsh(I_t)
    res.at[halo_i, 'lambda0'] = lambdas[0]
    res.at[halo_i, 'lambda1'] = lambdas[1]
    res.at[halo_i, 'lambda2'] = lambdas[2]

In [ ]:
res.head()

# Add $\sigma/v_\theta$ to the data frame

In [ ]:
tmp = gal_dynamics.loc[association.loc[res.index]['gal_id']]
res['sigma_over_vtheta'] = np.array(tmp['sigma_over_vtheta'])

# Plot all that

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter3D(res.lambda0, res.lambda1, res.lambda2)
ax.set_xlabel('$\lambda_0$')
ax.set_ylabel('$\lambda_1$')
ax.set_ylabel('$\lambda_2$')
ax.grid()

In [ ]:
res.head()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter3D(
    res.lambda0,
    res.lambda1,
    res.lambda2, label=r'$\sigma/v_\theta > 1$', alpha=0.5, c=res.sigma_over_vtheta)


ax.set_xlabel('$\lambda_0$')
ax.set_ylabel('$\lambda_1$')
ax.set_ylabel('$\lambda_2$')
ax.grid()
ax.legend()


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
tmp = res[res.sigma_over_vtheta > 1]
trace = [None, None]
trace[0] = ax.scatter(
    np.log(tmp.lambda1),
    np.log(tmp.lambda2), label=r'$\sigma/v_\theta > 1$', alpha=0.5, c='red')
tmp = res[res.sigma_over_vtheta <= 1]
trace[1] = ax.scatter(
     np.log(tmp.lambda1),
     np.log(tmp.lambda2), label=r'$\sigma/v_\theta \leq 1$', alpha=0.5, c='blue')


ax.set_xlabel('$\lambda_1$')
ax.set_ylabel('$\lambda_2$')
ax.grid()
ax.legend()


### $\lambda_2$ - $\lambda_1$ - $\sigma/v_\theta$

In [ ]:
fig = plt.figure()
ax11 = fig.add_subplot(121)
ax12 = fig.add_subplot(122, sharex=ax11, sharey=ax11)

tmp = res[res.sigma_over_vtheta > 1]
trace = [None, None]
trace[0] = ax11.scatter(
    tmp.lambda1,
    tmp.lambda2, label=r'$\sigma/v_\theta > 1$', alpha=0.5, c=tmp.lambda0)

tmp = res[res.sigma_over_vtheta <= 1]
trace[1] = ax12.scatter(
     tmp.lambda1,
     tmp.lambda2, label=r'$\sigma/v_\theta \leq 1$', alpha=0.5, c=tmp.lambda0)


#ax.set_xlabel('$\lambda_1$')
#ax.set_ylabel('$\lambda_2$')
ax11.grid()
ax12.grid()
ax11.legend()
ax12.legend()


### $\lambda_2$ - $\lambda_1$ - $\sigma/v_\theta$

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(
    np.log(res.lambda1),
    np.log(res.lambda2), 
    alpha=0.5, c=np.log(res.sigma_over_vtheta))


ax.set_xlabel('$\lambda_0$')
ax.set_ylabel('$\lambda_1$')
#ax.set_ylabel('$\lambda_2$')
ax.grid()
ax.legend()
#plt.colorbar()


### Corrélations
Recherche de corrélations entre $\lambda_i$ et $\sigma/_\theta$

In [ ]:
res[['lambda0', 'lambda1', 'lambda2', 'sigma_over_vtheta']].corr()

In [ ]:
#%timeit res['lambda_square'] = res.eval('sqrt(lambda0**2 + lambda1**2 + lambda2**2)')
res['lambda_square'] = np.sqrt(res.lambda0**2 + res.lambda1**2 + res.lambda2**2)

In [ ]:
res[['lambda_square', 'sigma_over_vtheta']].corr()

Aucune corrélation semblerait-il…

## KDE des $\lambda$

In [ ]:
plt.figure()
res['lambda0'].plot.kde(label='$\lambda_0$')
res['lambda1'].plot.kde(label='$\lambda_1$')
res['lambda2'].plot.kde(label='$\lambda_2$')
plt.legend()

In [ ]:
res.describe()

In [ ]:
from pandas.tools.plotting import scatter_matrix
plt.figure(figsize=(23, 23*9/16.))
ax = plt.gca()
scatter_matrix(res,
               alpha=0.5, diagonal='kde', ax=ax)
None

In [ ]:
from pandas.tools.plotting import scatter_matrix
plt.figure(figsize=(23, 23*9/16.))
logres = np.log10(np.sqrt(res.astype(np.float)))
ax = plt.gca()
scatter_matrix(logres[['lambda0', 'lambda1', 'lambda2', 'sigma_over_vtheta', 'lambda_square']],
               alpha=1, diagonal='kde', ax=ax)
None

## Rapport d'asymétrie
On travaille avec $$\tau = \frac{\lambda_2}{\lambda_1} \propto? \frac{\sigma}{v_\theta}$$

In [ ]:
res.sort_index()

In [ ]:
halo_list.loc[63792]

In [ ]:
res['tau'] = res.lambda2/res.lambda1
res['tau'].describe()

In [ ]:
res[['tau', 'sigma_over_vtheta']].corr()

In [ ]:
from pandas.tools.plotting import scatter_matrix
plt.figure(figsize=(23, 23*9/16.))
logres = np.log(res.astype(np.float))
ax = plt.gca()
scatter_matrix(logres[['lambda0', 'lambda1', 'lambda2', 'sigma_over_vtheta', 'lambda_square', 'tau']],
               alpha=0.5, diagonal='kde', ax=ax)
None

In [ ]:
plt.figure()
np.log10(res.lambda0).plot.hist(bins=20)

In [ ]:
res[res.lambda0 < 1e-15]